# Installing kube-oidc-proxy

Kube-oidc-proxy provides an OIDC authentication proxy to the Kubernetes API server.  
Used in conjunction with the kubelogin plugin for kubectl, this provides OIDC authentication
to kubectl to interact with Kubernetes clustes.

## Prerequisites
- [Learning Jupyter](../getting-started/learning_jupyter.ipynb)
- Install Jupyter bash kernel:
  - Windows... coming soon!
  - [Mac](../getting-started/mac/install_jupyter_bash_kernel.ipynb)
  - [Linux](../getting-started/linux/install_jupyter_bash_kernel.ipynb)
- [Install Kubernetes](../kubernetes/install_k8s_cluster.ipynb)
- Configure DNS
  - Mac: [Install DNSmasq](../dnsmasq/mac/install_dnsmasq.ipynb)
  - Linux: [Install DNSmasq](../../dnsmasq/linux/install_dnsmasq.ipynb)
  - Windows: Install Acrylic... coming soon!
- [Install Monitoring](../monitoring/install_kube-prometheus-stack.ipynb)
- [Install Logging](../logging/install_loki.ipynb)
- [Install cert-manager](../cert-manager/install_cert-manager.ipynb)
- [Install Ingress Controller](../ingress/install_ingress-nginx.ipynb)
- [Install Postgres](../postgres/install_postgres.ipynb)
- [Install OpenLDAP](../openldap/install_openldap.ipynb)
- [Install Keycloak](../keycloak/install_keycloak.ipynb)

### Clone the kube-oidc-proxy repository

Start by cloning the kube-oidc-proxy repository locally.  This repo contains a helm chart we will use below to install the application to our cluster.

In [1]:
git clone https://github.com/jetstack/kube-oidc-proxy.git

Cloning into 'kube-oidc-proxy'...
remote: Enumerating objects: 63260, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 63260 (delta 15), reused 34 (delta 13), pack-reused 63215
Receiving objects: 100% (63260/63260), 105.31 MiB | 13.13 MiB/s, done.
Resolving deltas: 100% (26405/26405), done.


### Install kube-oidc-proxy

Use helm to install the kube-oidc-proxy application.  For our demonstration, we are using `values.nephtek.yaml` that configures kube-oidc-proxy to talk to the Keycloak server we installed earlier, and to expose the proxy at `kube-api.k8s.nephtek.com`.  It uses the `nephtek-com-issuer` certificate issuer to create and manage the TLS certificate used by the ingress.

In [2]:
helm upgrade --install -n kube-oidc-proxy --create-namespace kube-oidc-proxy kube-oidc-proxy/deploy/charts/kube-oidc-proxy/ -f values.nephtek.yaml

Release "kube-oidc-proxy" does not exist. Installing it now.
NAME: kube-oidc-proxy
LAST DEPLOYED: Sun Oct  2 09:12:07 2022
NAMESPACE: kube-oidc-proxy
STATUS: deployed
REVISION: 1
NOTES:
1. Get the application URL by running these commands:
  export POD_NAME=$(kubectl get pods --namespace kube-oidc-proxy -l "app.kubernetes.io/name=kube-oidc-proxy,app.kubernetes.io/instance=kube-oidc-proxy" -o jsonpath="{.items[0].metadata.name}")
  echo "Visit http://127.0.0.1:8080 to use your application"
  kubectl port-forward $POD_NAME 8080:80


### Install roles & clusterroles

From here all that remains is to install roles and clusterroles to enable LDAP users and groups to gain access to the cluster.  

In [3]:
kubectl apply -k kustomize/overlays/rancher-desktop/

clusterrolebinding.rbac.authorization.k8s.io/oidc-admin-group created
ingress.networking.k8s.io/kube-oidc-proxy created


### Install oidc-login kubectl plugin

Since the OIDC standard is a web-based login standard, kubectl needs a plugin in order to defer login and
token refresh requests to the browser.  The `oidc-login` kubectl plugin, also known as `kubelogin`, provides
these features.

We will use the `krew` plugin manager for kubectl to install the `oidc-login` plugin.  Start by installing `krew` 
using the instructions found [here](https://krew.sigs.k8s.io/docs/user-guide/setup/install/).

Once `krew` is installed, we can use it to install the `oidc-login` plugin:

In [4]:
kubectl krew install oidc-login

Updated the local copy of plugin index.
  New plugins available:
    * accurate
    * applier
    * aws-auth
    * cnpg
    * colorize-applied
    * confirm
    * count
    * datree
    * dds
    * directpv
    * discover
    * dtlogin
    * execws
    * explore
    * foreach
    * htpasswd
    * ice
    * ingress-rule
    * kadalu
    * kc
    * kruise
    * ktop
    * kuota-calc
    * linstor
    * liqo
    * log2rbac
    * moco
    * nsenter
    * print-env
    * pv-migrate
    * realname-diff
    * relay
    * rename-pvc
    * safe
    * secretdata
    * stern
    * strace
    * switch-config
    * volsync
    * vpa-recommendation
    * windows-debug
  Upgrades available for installed plugins:
    * gadget v0.3.1 -> v0.8.0
    * krew v0.4.2 -> v0.4.3
    * kyverno v1.5.1 -> v1.7.4
    * lineage v0.4.1 -> v0.5.0
    * minio v4.3.1 -> v4.5.1
    * oidc-login v1.25.0 -> v1.25.3
    * rabbitmq v1.10.0 -> v2.0.0
    * sniff v1.6.1 -> v1.6.2
    * tree v0.4.1 -> v0.4.2
    * tunnel v1.4.

### Finished
